# Importing libraris

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pandas as pd
import os


In [2]:

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

#New Add
landmark_points_68 = [1,162,234,93,58,172,136,149,148,152,377,378,365,397,288,323,454,389,71,63,105,66,107,336,
                  296,334,293,301,168,197,5,4,75,97,2,326,305,33,160,158,133,153,144,362,385,387,263,373,
                  380,61,39,37,0,267,269,291,405,314,17,84,181,78,82,13,312,308,317,14,87]
#Creating cols names as list 
colsx=  ['x' + str(i + 1) for i in range(468)]
colsy=  ['y' + str(i + 1) for i in range(468)]
imgnolist=[]

# creating empty dataframe to append points on it 

dfx=pd.DataFrame()
dfy=pd.DataFrame()


#path= "D:\ITI Material\ML 1 Project\Head Pose Estimation ML Project-20220615T153744Z-001\Head Pose Estimation ML Project\Dataset\Images\Images2"
path='/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Images'

for img in glob.glob(path+'/*.jpg'):
   
    image=cv2.imread(img,0)  
    

        
    # Flip the image horizontally for a later selfie-view display
    # Also convert the color space from BGR to RGB
        
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

    # To improve performance
    image.flags.writeable = False
        
        # Get the result
    results = face_mesh.process(image)
        
    # To improve performance
    image.flags.writeable = True
        
        # Convert the color space from RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = image.shape
    x_points=[]
    y_points=[]
    face_2d = []
    
    if results.multi_face_landmarks:
        #1. save mat file no
        img_no, _ = os.path.splitext(os.path.basename(img))
        imgnolist.append(img_no)
        
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                # if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199 :
                    if idx == 1:
                        nose_2d = (lm.x * img_w, lm.y * img_h)


                    x, y = int(lm.x * img_w), int(lm.y * img_h)

                    # Get the 2D Coordinates
                    x_points.append(x)
                    y_points.append(y)
                    
            # convert the list to numpy array to convert it easy to dataframe 
            x_points=np.array(np.transpose(x_points).astype('float'),dtype=np.float32).reshape(1,-1)
            y_points=np.array(np.transpose(y_points).astype('float'),dtype=np.float32).reshape(1,-1)
            dfx2=pd.DataFrame(x_points)
            dfy2=pd.DataFrame(y_points)
        # concating the current points of current image with the previous image points 
        dfx = pd.concat([dfx,dfx2],axis=0)
        dfy = pd.concat([dfy,dfy2],axis=0)
 


NameError: name 'glob' is not defined

In [ ]:

# df68 = pd.DataFrame(pd.concat([ dfx,dfy ],axis=1))
# df68.to_csv('D:\ITI Material\ML 1 Project\Head Pose Estimation ML Project-20220615T153744Z-001\Head Pose Estimation ML Project\Dataset\data468.csv',index=False)


## Checking the final dataframes before saving 

In [ ]:
dfx.info()

In [ ]:
dfx.describe()

In [ ]:
dfx

In [ ]:
%config Completer.use_jedi = False

In [ ]:
dfx.columns=colsx

In [ ]:
dfx

In [ ]:
dfy.columns=colsy

In [ ]:
dfy

In [ ]:
dfx.to_csv('D:\ITI Material\ML 1 Project\Head Pose Estimation ML Project-20220615T153744Z-001\Head Pose Estimation ML Project\Dataset\data_x_468.csv',index=False)
dfy.to_csv('D:\ITI Material\ML 1 Project\Head Pose Estimation ML Project-20220615T153744Z-001\Head Pose Estimation ML Project\Dataset\data_y_468.csv',index=False)

In [ ]:

df_media = pd.DataFrame(pd.concat([ dfx,dfy ],axis=1))
df_media['image_name'] = imgnolist
#df_media.to_csv('D:\ITI Material\ML 1 Project\Head Pose Estimation ML Project-20220615T153744Z-001\Head Pose Estimation ML Project\Dataset\data_media.csv',index=False)

In [ ]:
df_media

In [4]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pandas
import os
import cv2
import mediapipe as mp
import numpy as np
import time
import pickle
from math import cos, sin
from sklearn.preprocessing import MinMaxScaler,StandardScaler

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
pkl_filename='model_Rm.pkl'
with open(pkl_filename, 'rb') as file:
    pretrained_model = pickle.load(file)

mp_drawing = mp.solutions.drawing_utils

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):

    yaw = -yaw
    if tdx != None and tdy != None:
        tdx = tdx
        tdy = tdy
    else:
        height, width = img.shape[:2]
        tdx = width 
        tdy = height

    # X-Axis pointing to right. drawn in red
    x1 = size * (cos(yaw) * cos(roll)) + tdx
    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy

    # Y-Axis | drawn in green
    #        v
    x2 = size * (-cos(yaw) * sin(roll)) + tdx
    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy

    # Z-Axis (out of the screen) drawn in blue
    x3 = size * (sin(yaw)) + tdx
    y3 = size * (-cos(yaw) * sin(pitch)) + tdy

    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)
    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)

    return img
def yaw2rotmat(yaw,pitch,roll):
    x = roll
    y = pitch
    z = yaw
    ch = np.cos(z)
    sh = np.sin(z)
    ca = np.cos(y)
    sa = np.sin(y)
    cb = np.cos(x)
    sb = np.sin(x)
    rot = np.zeros((3,3), 'float64')
    rot[0][0] = ch * ca
    rot[0][1] = sh*sb - ch*sa*cb
    rot[0][2] = ch*sa*sb + sh*cb
    rot[1][0] = sa
    rot[1][1] = ca * cb
    rot[1][2] = -ca * sb
    rot[2][0] = -sh * ca
    rot[2][1] = sh*sa*cb + ch*sb
    rot[2][2] = -sh*sa*sb + ch*cb
    return rot

cap = cv2.VideoCapture(0)
i = 0
while cap.isOpened():
    success, image = cap.read()

    start = time.time()

    # Flip the image horizontally for a later selfie-view display
    # Also convert the color space from BGR to RGB
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

    # To improve performance
    image.flags.writeable = False
    
    # Get the result
    results = face_mesh.process(image)
    
    # To improve performance
    image.flags.writeable = True
    
    # Convert the color space from RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    img_h, img_w, img_c = image.shape
    face_3d = []
    face_2d = []
    x_points=[]
    y_points=[]
 
 

    if results.multi_face_landmarks:
        
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                # if idx == 33 or idx == 263 or idx == 1 or idx == 61 or idx == 291 or idx == 199:
                    if idx == 1:
                        nose_2d = (lm.x * img_w, lm.y * img_h)
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)

                    x, y = int(lm.x * img_w), int(lm.y * img_h)

                    # Get the 2D Coordinates
                    x_points.append(x)
                    y_points.append(y)
                    face_2d.append([x, y])

                    # Get the 3D Coordinates
                    face_3d.append([x, y, lm.z])       
        # Convert it to the NumPy array
        face_2d = np.array(face_2d, dtype=np.float64)
        x_point=np.array(np.transpose(x_points).astype('float'),dtype=np.float64).reshape(1,-1)
        y_point=np.array(np.transpose(y_points).astype('float'),dtype=np.float64).reshape(1,-1)
        new_data = np.concatenate((x_point,y_point),axis=1)
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(new_data)
        i+=1
        print(f"**********************************\n frame {i}\n {face_2d[0]}\n**************************************")
        # Convert it to the NumPy array
        face_3d = np.array(face_3d, dtype=np.float64)

        # The camera matrix
        focal_length = 1 * img_w

        cam_matrix = np.array([ [focal_length, 0, img_h / 2],
                                [0, focal_length, img_w / 2],
                                [0, 0, 1]])

        # # The distortion parameters
        dist_matrix = np.zeros((4, 1), dtype=np.float64)

        # # Solve PnP
        success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)
        
        # # Get rotational matrix
        rmat, jac = cv2.Rodrigues(rot_vec)

        # Load pretrained model
        
        angles = pretrained_model.predict(x_scaled)
        print(angles)
        x = angles[0][0] *360 
        y = angles[0][1] *360
        z = angles[0][2] *360
        

        # image = draw_axis(image,x,y,z)
        # # Get angles
        # angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)
        # axis = np.float32([[0.0, 0.0, 0.0], 
        #                  [0.0, 0.0, 0.0], 
        #                  [0.0, 0.0, 0.5]])
        # x = angles[0] *360 
        # y = angles[1] *360
        # z = angles[2] *360
        
        # tvec = np.array([0.0, 0.0, 1.0], np.float64) # translation vector

        # rot_matrix = yaw2rotmat(-x,-y,-z)
        # rvec, jacobian = cv2.Rodrigues(rot_matrix)
        # imgpts, jac = cv2.projectPoints(axis, rvec, tvec, cam_matrix, dist_matrix)
        # # # Get the y rotation degree
        # x = angles[0] * 360
        # y = angles[1] * 360
        # z = angles[2] * 360
        
        
        # See where the user's head tilting
        if y < -10:
            text = "Looking Left"
        elif y > 10:
            text = "Looking Right"
        elif x < -10:
            text = "Looking Down"
        elif x > 10:
            text = "Looking Up"
        else:
            text = "Forward"

        # Display the nose direction
        # nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)

        p1 = (int(nose_2d[0]), int(nose_2d[1]))
        p2 = (int(nose_2d[0] + y * 10) , int(nose_2d[1] - x * 10))
        # # p2 = (int(imgpts[2][0][0]), int(imgpts[2][0][1]))
        cv2.line(image, p1, p2, (255, 0, 0), 1)

        # Add the text on the image
        cv2.putText(image, text, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
        cv2.putText(image, "x: " + str(np.round(x,2)), (500, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(image, "y: " + str(np.round(y,2)), (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(image, "z: " + str(np.round(z,2)), (500, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


        end = time.time()
        totalTime = end - start

        fps = 1 / totalTime
        print("FPS: ", fps)

        cv2.putText(image, f'FPS: {int(fps)}', (20,450), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

        mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec)


    cv2.imshow('Head Pose Estimation', image)

    if cv2.waitKey(5) & 0xFF == 27:
        break


cap.release()

**********************************
 frame 1
 [312. 366.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  18.265090251921528
**********************************
 frame 2
 [326. 383.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.524227631797995
**********************************
 frame 3
 [328. 392.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  19.577321079335146
**********************************
 frame 4
 [326. 392.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.816086514244063
**********************************
 frame 5
 [323. 392.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  21.970864784733607
**********************************
 frame 6
 [307. 386.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.72965913401615
**********************************


[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  19.69637658019798
**********************************
 frame 54
 [296. 414.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.93485856768683
**********************************
 frame 55
 [296. 414.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  23.53771991357782
**********************************
 frame 56
 [297. 413.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.354241615102143
**********************************
 frame 57
 [296. 413.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.352812018695275
**********************************
 frame 58
 [296. 412.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.554265587610573
**********************************
 frame 59
 [296. 414.]
**************************************
[[ 1.14620982 -3.94010639 -0.66

FPS:  23.35384581120057
**********************************
 frame 105
 [294. 417.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  23.295476761752422
**********************************
 frame 106
 [291. 417.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  23.889095196327474
**********************************
 frame 107
 [291. 417.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.925581980071385
**********************************
 frame 108
 [292. 418.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  23.33370792142551
**********************************
 frame 109
 [292. 416.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  20.744061367109644
**********************************
 frame 110
 [291. 416.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.547960670261322

FPS:  23.670154290680482
**********************************
 frame 157
 [283. 410.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  23.321771525480276
**********************************
 frame 158
 [282. 410.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  20.62461399264373
**********************************
 frame 159
 [282. 409.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  19.17045568810275
**********************************
 frame 160
 [283. 409.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.668115072609453
**********************************
 frame 161
 [282. 408.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.409061281188226
**********************************
 frame 162
 [282. 408.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.801078542228407

[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  21.250647251650427
**********************************
 frame 208
 [240. 343.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  23.33189072521653
**********************************
 frame 209
 [228. 345.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  23.333837731986293
**********************************
 frame 210
 [203. 357.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.411935066739336
**********************************
 frame 211
 [203. 361.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.713288530999712
**********************************
 frame 212
 [208. 366.]
**************************************
[[ 1.14620982 -3.94010639 -0.66143423]]
FPS:  22.35400333633568
